In [ ]:
!pip install --upgrade gensim

In [ ]:
! unzip "/content/drive/MyDrive/Salinan idwiki_word2vec_300.zip" -d "/content/"

In [ ]:
import gensim
import  numpy as np

path = '/content/idwiki_word2vec_300.model'
word2vec = gensim.models.word2vec.Word2Vec.load(path)
print(word2vec.most_similar('raja'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('rajanya', 0.6927156448364258), ('kerajaan', 0.6887479424476624), ('penguasa', 0.6561841368675232), ('maharaja', 0.6353377103805542), ('firaun', 0.6156154870986938), ('kertanagara', 0.6069246530532837), ('purnawarman', 0.6000310182571411), ('ratu', 0.5888345241546631), ('permaisuri', 0.5711089372634888), ('takhta', 0.5686213970184326)]


In [ ]:
word2vec.wv.syn0.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


(331792, 300)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

import numpy as np
import pandas as pd

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset_clean_label.csv',header=0, names=['sentimen','text'])
df.head()

,sentimen,text
0,1,ini cukup bagus sih untuk cream blush dengan h...
1,1,well lumayan sih ini tidak terlalu pigmented t...
2,1,produk ini cocok banget sih dipake buat make u...
3,1,blush liquid cinta sudah murah terus hasil bag...
4,1,hmm suka nya dia kecil dan ringan tetapi ini m...


In [ ]:
text = df['text'].tolist()
print(len(text))

11084


In [ ]:
y = df['sentimen']
y = to_categorical(y)
print(y)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [ ]:
df['sentimen'].value_counts()

1    8089
0    1666
2    1329
Name: sentimen, dtype: int64

In [ ]:
token = Tokenizer(oov_token='<OOV>')
token.fit_on_texts(text)

In [ ]:
token.index_word

{1: '<OOV>',
 2: 'saya',
 3: 'ini',
 4: 'tidak',
 5: 'dan',
 6: 'yang',
 7: 'pakai',
 8: 'di',
 9: 'banget',
 10: 'tetapi',
 11: 'warna',
 12: 'bibir',
 13: 'jadi',
 14: 'karena',
 15: 'juga',
 16: 'kalau',
 17: 'bikin',
 18: 'begitu',
 19: 'buat',
 20: 'sangat',
 21: 'suka',
 22: 'nya',
 23: 'sih',
 24: 'kulit',
 25: 'sama',
 26: 'bagus',
 27: 'cocok',
 28: 'saja',
 29: 'harga',
 30: 'produk',
 31: 'kering',
 32: 'lama',
 33: 'untuk',
 34: 'pas',
 35: 'sudah',
 36: 'dia',
 37: 'muka',
 38: 'sunscreen',
 39: 'liptint',
 40: 'emina',
 41: 'ada',
 42: 'beli',
 43: 'tekstur',
 44: 'lebih',
 45: 'agak',
 46: 'itu',
 47: 'terus',
 48: 'enak',
 49: 'lip',
 50: 'bisa',
 51: 'blush',
 52: 'lagi',
 53: 'lumayan',
 54: 'ya',
 55: 'pertama',
 56: 'ke',
 57: 'dari',
 58: 'tahan',
 59: 'kurang',
 60: 'wangi',
 61: 'murah',
 62: 'coba',
 63: 'minyak',
 64: 'dipake',
 65: 'terlalu',
 66: 'repurchase',
 67: 'gampang',
 68: 'liat',
 69: 'turut',
 70: 'mau',
 71: 'seperti',
 72: 'banyak',
 73: 'telah',


In [ ]:
vocab = len(token.index_word)+1
vocab

13446

In [ ]:
x = ['bagus banget']
token.texts_to_sequences(x)

[[26, 9]]

In [ ]:
encode_text = token.texts_to_sequences(text)

In [ ]:
max_kata=100
X = pad_sequences(encode_text, maxlen=max_kata, padding="post")
X

array([[    3,   107,    26, ...,     0,     0,     0],
       [  864,    53,    23, ...,     0,     0,     0],
       [   30,     3,    27, ...,     0,     0,     0],
       ...,
       [  251, 13442,  2287, ...,     0,     0,     0],
       [  119,     9,     9, ...,     0,     0,     0],
       [  425,   551,     3, ...,     0,     0,     0]], dtype=int32)

In [ ]:
embeding_vector_length = 300
embeding_vector_length=word2vec.wv.syn0.shape[1]
embeding_matrix=np.zeros((vocab,embeding_vector_length))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#build embedding index
for word,i in sorted(token.word_index.items(),key=lambda x:x[1]):
    if i>vocab:
        break
    if word in word2vec.wv.vocab:
        embeding_vector=word2vec.wv[word]
        embeding_matrix[i]=embeding_vector

In [ ]:
# Just to have a glance at the vector for our words 
embeding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.87361014,  2.59380031, -1.86611855, ..., -0.60246372,
         0.52344346,  0.76725423],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.48136151, -0.20923139,  0.93265611, ..., -0.56763309,
        -0.54428643,  1.35350811]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42, test_size= 0.25)

In [ ]:
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

In [ ]:
vec_size =300
opt = SGD(lr=0.008)
model = Sequential()
model.add(Embedding(vocab, embeding_vector_length, weights=[embeding_matrix], input_length=max_kata, trainable=False))

model.add(Conv1D(128,3, strides=1,padding="valid"))
model.add(MaxPooling1D())

model.add(Dropout(0.5))

model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))

model.add(Dropout(0.2))
model.add(Dense(16, activation="relu"))


model.add(GlobalMaxPooling1D())
model.add(Dense(3, activation="softmax"))
model.summary()


In [ ]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=opt)

In [ ]:
history = model.fit(X_train, y_train, epochs=130, validation_data=(X_test, y_test), batch_size=20)

In [ ]:
print(history.history['loss'])
print(history.history['accuracy'])
print(history.history['val_loss'])
print(history.history['val_accuracy'])

In [ ]:
from matplotlib import pyplot

In [ ]:
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_test2=np.array(X_test)
y_test2=np.array(y_test)
y_pred=model.predict(X_test2)
hasilPredict = (y_pred >= 0.5).astype(int)
from sklearn.metrics import classification_report
print(classification_report(y_test2,hasilPredict))